In [ ]:
!pip install -U bitsandbytes accelerate transformers

# 데이터셋

In [ ]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")
dataset['train'][0]

In [ ]:
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    GenerationConfig,
)
import torch

model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1"
)

doc = dataset["train"]['question'][0]
messages = [{"role": "user", "content": doc}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
)

gen_cfg = GenerationConfig(
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.to(model.device), 
        generation_config=gen_cfg,
    )
    print(tokenizer.decode(outputs[0].cpu().numpy()))